In [1]:
import requests
from retry import retry
import logging
logging.basicConfig()
#logger = logging.getLogger(__name__)


TEMPORARY_ERROR_CODES = (400, 500, 502, 503, 504) # 一時的なエラーを表すステータスコード


def main():
    """
    メインとなる処理
    """
    response = fetch('http://httpbin.org/status/200,400,404,500,502,503,504')
    if 200 <= response.status_code < 300:
        print('Success!')
    else:
        print('Error!')
        
        
@retry(tries=4, delay=1, backoff=2) # exceptions=Exception, , logger='logger'
def fetch(url):
    """
    指定したURLを取得してResponseオブジェクトを返す
    一時的なエラーが起きた場合は、最大3回リトライする
    """
    print('Retrieving{0}...'.format(url))
    response = requests.get(url)
    print('Status: {0}'.format(response.status_code))
    if response.status_code not in TEMPORARY_ERROR_CODES:
        return response # 一時的なエラーでなければresponseを返す
    
    # 一時的なエラーの場合は例外を発生させてリトライする
    raise Exception('Temporary Error: {0}'.format(response.status_code))
    
    
main()

In [35]:
%%writefile logging_test_lib.py

import pandas as pd
import numpy as np
from retry import retry
import logging


@retry(tries=3, delay=1, backoff=2) # exceptions=Exception, , logger='logger'
def fetch():
    a = np.random.randint(0,3)
    url = ['http://httpbin.org/status/200,404,503', 
             'https://stocks.finance.yahoo.co.jp/stocks/detail/?code=1000.T',
             'https://stocks.finance.yahoo.co.jp/stocks/detail/?code=1301.T']

    tables = pd.read_html(url[a], header=0)
    
    return tables

Overwriting logging_test_lib.py


In [33]:
%%writefile logging_test.py

import logging
import logging_test_lib


def main():
    # logging.basicConfig()
    logging.basicConfig(filename='logging_test.log', level=logging.INFO)
    
    logging.info('Started')

    result = []

    try:
        result = logging_test_lib.fetch()

    except Exception as e:
        logging.warning(e)
        print(e)
        print('failed')

    print('finish')
    print(result)
    
    logging.info('Finished')
    

if __name__ == '__main__':
    main()

Overwriting logging_test.py


In [41]:
!python logging_test.py

finish
[Empty DataFrame
Columns: [(株)極洋, Unnamed: 1, 3,665, 前日比-10（-0.27%）]
Index: [],    売気配  株価  買気配  Unnamed: 3 Unnamed: 4
0  --- NaN  ---         NaN        NaN
1  NaN NaN  ---         NaN        ---,     グローバル･スマート･イノベーション(年1回)その他リターン(1年)47.34%  \
0  先進国資本エマージング株式ファンド(1年決算)国際株式リターン(1年)23.06%   
1  ピムコ･ストラテジック･インカム(H有/3カ月)国際債券リターン(1年)4.65%   

          ファンド･マネジャー(国内株式)国内株式リターン(1年)28.83%  
0         インド債券オープン(毎月決算型)国際債券リターン(1年)19.58%  
1  ピムコ･ストラテジック･インカム(H有/年1回)国際債券リターン(1年)4.63%  , Empty DataFrame
Columns: [値上がり率1933社 1. 郵船ロジ+29.47％ 2. ＵＭＮ+26.85％ 3. テクファＨ+22.66％ もっと見る, 値下がり率1654社 1. ヒーハイス-20.35％ 2. ぐるなび-16.37％ 3. 五洋インテ-15.45％ もっと見る]
Index: []]
